In [1]:
import pandas as pd

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

### Load NHTSA and Ford AsBuilt Data for vehicles

Filtered to only include the Escape for this example

In [2]:
from lookup import lookup, print_breakdown

df_nhtsa = lookup(
  searches=['Bronco Sport', 'Escape'],
  include_openpilot=True,
  include_police=False,
  min_model_year=2019,
)

print()
print_breakdown(df_nhtsa)

Loaded 78 VINs (filter_comment='Bronco Sport', include_openpilot=True, skipped=61)
Loaded AsBuilt data for 78 VINs
Loaded 283 VINs (filter_comment='Escape', include_openpilot=True, skipped=61)
Loaded AsBuilt data for 283 VINs


Decoding VINs: 100%|██████████| 361/361 [00:00<00:00, 17157.05it/s]

Decoded 361 VINs

Model         ModelYear  Series         
Bronco Sport  2021       BIG BEND            3
                         OUTER BANKS         2
              2022       BIG BEND            2
                         OUTER BANKS         1
              2023       BADLANDS            9
                         BIG BEND           12
                         HERITAGE            4
                         OUTER BANKS        13
              2024       BIG BEND           19
                         OUTER BANKS        13
Escape        2019       S                   9
                         SE                  8
                         SEL                10
                         Titanium            1
              2020       S                  15
                         SE                 28
                         SE FHEV             7
                         SEL                30
                         Titanium            5
                         Titanium FHEV       8
 

In [3]:
df_nhtsa.head()

,ABS,ActiveSafetySysNote,AdaptiveCruiseControl,AdaptiveDrivingBeam,AirBagLocFront,AirBagLocKnee,AirBagLocSide,AutoReverseSystem,AutomaticPedestrianAlertingSound,Axles,BatteryKWh,BatteryType,BlindSpotMon,BodyClass,BrakeSystemType,BusType,CAN_AACN,CIB,DaytimeRunningLight,DisplacementCC,DisplacementCI,DisplacementL,Doors,DriveType,DynamicBrakeSupport,EDR,ESC,ElectrificationLevel,EngineConfiguration,EngineCylinders,EngineHP,EngineManufacturer,EngineModel,ForwardCollisionWarning,FuelInjectionType,FuelTypePrimary,FuelTypeSecondary,GVWR,KeylessIgnition,LaneCenteringAssistance,LaneDepartureWarning,LaneKeepSystem,LowerBeamHeadlampLightSource,Make,MakeID,Manufacturer,ManufacturerId,Model,ModelID,ModelYear,NCSABodyType,NCSAMake,NCSAModel,Note,OtherEngineInfo,ParkAssist,PedestrianAutomaticEmergencyBraking,PlantCity,PlantCompanyName,PlantCountry,PlantState,RearAutomaticEmergencyBraking,RearCrossTrafficAlert,RearVisibilitySystem,SeatBeltsAll,SeatRows,Seats,SemiautomaticHeadlampBeamSwitching,Series,SteeringLocation,TPMS,TopSpeedMPH,TractionControl,TransmissionSpeeds,TransmissionStyle,Turbo,VIN,VehicleDescriptor,VehicleType,WheelBaseShort,WheelSizeFront,WheelSizeRear,Wheels
0,,,,,1st Row (Driver and Passenger),Driver Seat Only,1st and 2nd Rows,,,,,,,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,,Not Applicable,,,,1500.0,91.53561614209,1.5,4,4WD,,,,,In-Line,3,,Ford,DRAGON,,,Gasoline,,"Class 1C: 4,001 - 5,000 lb (1,814 - 2,268 kg)",,,,,,FORD,460,"FORD MOTOR COMPANY, MEXICO",979,Bronco Sport,27411,2024,"Utility Vehicle, Unknown body type",Ford,Other (light truck),,,,,HERMOSILLO,,MEXICO,SONORA,,,,Manual,,,,BIG BEND,,Direct,,,,,,3FMCR9B64RRE15114,3FMCR9B6*RR,MULTIPURPOSE PASSENGER VEHICLE (MPV),,,,
1,Standard,,,,1st Row (Driver and Passenger),Driver Seat Only,1st and 2nd Rows,Standard,,,,,Standard,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,Hydraulic,Not Applicable,,Standard,Standard,1500.0,91.53561614209,1.5,4,RWD,Standard,Standard,Standard,,,,,Ford,DRAGON GTDI,Standard,,Gasoline,,"Class 1C: 4,001 - 5,000 lb (1,814 - 2,268 kg)",Standard,,Standard,Standard,,FORD,460,FORD MOTOR COMPANY,976,Escape,1798,2023,"Compact Utility (Utility Vehicle Categories ""S...",Ford,Escape,,,,Standard,LOUISVILLE,,UNITED STATES (USA),KENTUCKY,Optional,Standard,Standard,Manual,2,5,Standard,ST Line,,Direct,,Standard,8,Automatic,Yes,1FMCU0MN4PUA27630,1FMCU0MN*PU,MULTIPURPOSE PASSENGER VEHICLE (MPV),,,,
2,Standard,,Optional,,1st Row (Driver and Passenger),Driver Seat Only,1st and 2nd Rows,Standard,,,,,Standard,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,Hydraulic,Not Applicable,Standard,Standard,Standard,1500.0,91.53561614209,1.5,4,RWD,,Standard,Standard,,In-Line,3,181,Ford,Dragon DI TC,Standard,,Gasoline,,"Class 1C: 4,001 - 5,000 lb (1,814 - 2,268 kg)",Standard,Optional,Standard,Standard,Halogen,FORD,460,FORD MOTOR COMPANY,976,Escape,1798,2022,"Compact Utility (Utility Vehicle Categories ""S...",Ford,Escape,GVWR: 4360-4920 lbs,,,,LOUISVILLE,,UNITED STATES (USA),KENTUCKY,,Standard,Standard,Manual,2,5,Standard,SE,,Direct,,Standard,8,Automatic,Yes,1FMCU0G61NUB21501,1FMCU0G6*NU,MULTIPURPOSE PASSENGER VEHICLE (MPV),,,,
3,Standard,,,,1st Row (Driver and Passenger),Driver Seat Only,1st and 2nd and 3rd Rows,Standard,,2,,,Standard,Sport Utility Vehicle (SUV)/Multi-Purpose Vehi...,Hydraulic,Not Applicable,,,Standard,1500.0,91.53561614209,1.5,4,RWD,,,Standard,,In-Line,3,,Ford,Dragon DI TC,Standard,,Gasoline,,"Class 1C: 4,001 - 5,000 lb (1,814 - 2,268 kg)",Standard,,Standard,Standard,LED,FORD,460,FORD MOTOR COMPANY,976,Escape,1798,2021,"Compact Utility (Utility Vehicle Categories ""S...",Ford,Escape,GVWR (lbs): 4360-4860,,Optional,Standard,LOUISVILLE,,UNITED STATES (USA),KENTUCKY,,Standard,Standard,Manual,2,5,Standard,S,Left-Hand Drive (LHD),Direct,,Standard,8,Automatic,,1FMCU0F66MUA48965,1FMCU0F6*MU,MULTIPURPOSE PASSENGER VEHICLE (MPV),106.70,17,17,4
4,Standard,,,,1st Row (Driver and Passenger),Driver Seat Only,1st and 2nd Rows,Standard,,,,,Standard,Sport Utility Veh

### Reduce to interesting columns

The NHTSA data has lots of columns, so we can filter down to only include the ones which don't change or aren't interesting to us.

In [4]:
SKIP = [
  'AirBagLocKnee',
  'AirBagLocSide',
  'DisplacementCC',
  'DisplacementCI',
  'EngineCylinders',
  'GVWR',
  'LowerBeamHeadlampLightSource',
  'Make',
  'MakeID',
  'Manufacturer',
  'ManufacturerId',
  'ModelID',
  'NCSABodyType',
  'NCSAMake',
  'NCSAModel',
  'PlantCity',
  'PlantCompanyName',
  'PlantCountry',
  'PlantState',
  'Seats',
  'WheelSizeFront',
  'WheelSizeRear',
  'VIN',
  'VehicleDescriptor',
]

KEEP = [
  'DisplacementL',
  # 'ElectrificationLevel',
  # 'FuelTypePrimary',
  # 'FuelTypeSecondary',
  'Series',
  'Trim',
]

def simplify_electrification_level(row):
  # TODO: debug 'HEV'
  return {
    '': 'ICE',
    'HEV (Hybrid Electric Vehicle) - Level Unknown': 'HEV',
    'PHEV (Plug-in Hybrid Electric Vehicle)': 'PHEV',
    'Strong HEV (Hybrid Electric Vehicle)': 'FHEV',
  }.get(row['ElectrificationLevel'], row['ElectrificationLevel'])

def simplify_series(row):
  series = row['Series']
  if series in ['SE FHEV', 'SE PHEV']:
    return 'SE'
  if series in ['SEL FHEV', 'SEL PHEV']:
    return 'SEL'
  if series in ['Titanium FHEV', 'Titanium PHEV']:
    return 'Titanium'
  # if series in ['ST Line Elite', 'ST Line Premium', 'ST Line Select']:
  #   return 'ST Line'
  return series

TRANSFORM_PROPERTIES = {
  'ElectrificationLevel': simplify_electrification_level,
  'Series': simplify_series,
}

properties = {}

for col in df_nhtsa.columns:
  if col in SKIP:
    continue

  if col in TRANSFORM_PROPERTIES:
    df_nhtsa[col] = df_nhtsa.apply(TRANSFORM_PROPERTIES[col], axis=1)

  property_values = set(df_nhtsa[col].unique())
  if '' in property_values and col not in KEEP:
    continue
  if len(property_values) == 1:
    continue

  properties[col] = property_values

df_nhtsa.drop(columns=['FuelTypePrimary', 'FuelTypeSecondary'], inplace=True, errors='ignore')
properties.pop('FuelTypePrimary', None)
properties.pop('FuelTypeSecondary', None)

properties

{'DisplacementL': {1.5, 2.0, 2.5},
 'DriveType': {'4WD', 'RWD'},
 'ElectrificationLevel': {'FHEV', 'HEV', 'ICE', 'PHEV'},
 'Model': {'Bronco Sport', 'Escape'},
 'ModelYear': {2019, 2020, 2021, 2022, 2023, 2024},
 'Series': {'Active',
  'BADLANDS',
  'BIG BEND',
  'Base',
  'HERITAGE',
  'OUTER BANKS',
  'PHEV',
  'S',
  'SE',
  'SEL',
  'ST Line',
  'ST Line Elite',
  'ST Line Premium',
  'ST Line Select',
  'Titanium'}}

### Combine NHTSA and Ford AsBuilt Data

We fetch the factory part numbers (software and hardware) from the Ford AsBuilt data and combine it with the NHTSA data.

In [5]:
from panda.python.uds import DATA_IDENTIFIER_TYPE

from asbuilt import AsBuiltData
from ecu import FordEcu
from settings import VehicleSetting, VehicleSettings


df_fw = df_nhtsa.copy()

# Drop columns that we don't care about (not in the properties)
df_fw.drop(
  columns=[col for col in df_nhtsa.columns if col not in properties and col != 'VIN'],
  inplace=True,
)


ecus = {
  'abs': FordEcu.AntiLockBrakeSystem,
  # 'engine': FordEcu.PowertrainControlModule,
  'eps': FordEcu.PowerSteeringControlModule,
  'fwdCamera': FordEcu.ImageProcessingModuleA,
  'fwdRadar': FordEcu.CruiseControlModule,
}


def get_ecu_identifier(ecu: FordEcu, identifier: int):
  def apply(row):
    data = AsBuiltData.from_vin(row['VIN'])
    if ecu not in data.ecus:
      return ''
    return data.get_identifier(ecu, identifier)
  return apply


def get_setting(setting: VehicleSetting):
  def apply(row):
    ecu = setting.ecu
    ecu_name = next(filter(lambda name: ecus[name] == ecu, ecus.keys()))
    ecu_part = row[f'{ecu_name}_part']
    if not ecu_part:
      return 'Missing ECU'
    # TODO: detect ECUs which we can't read
    # if ecu_part[0] in ('M', 'N', 'O', 'P'):
    #   return 'CAN FD'
    data = AsBuiltData.from_vin(row['VIN'])
    if ecu not in data.ecus:
      return 'Missing ECU'
    return data.get_setting_value(setting)
  return apply


# Add the ECU identifiers
for name, ecu in ecus.items():
  df_fw[f'{name}_fw'] = df_fw.apply(
    get_ecu_identifier(
      ecu, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER
    ),
    axis=1,
  )
  df_fw[f'{name}_part'] = df_fw.apply(get_ecu_identifier(ecu, 0xF111), axis=1)


# Determine platform (pre-Q3, Q3 or Q4)
def get_platform(row):
  camera_pn = row['fwdCamera_part']
  if '14H107' in camera_pn:
    return 'Q4'
  if not camera_pn:
    return 'Missing ECU'
  assert '14F403' in camera_pn

  model = row['Model']
  model_year = row['ModelYear']
  if model == 'Escape':
    if model_year < 2020:
      return 'Pre-Q3'
    if model_year < 2023:
      return 'Q3'

    # expected to hit the 14H107 check above
    assert False, 'Unhandled model year'

  if model == 'Bronco Sport':
    return 'Q3'

  assert False, 'Unhandled platform'


df_fw['Platform'] = df_fw.apply(get_platform, axis=1)


# Add settings
settings = {
  # TODO: read this from multiple modules to check that it's consistent
  'acc': VehicleSettings.ipma_enable_adaptive_cruise,
  'lca': VehicleSettings.ipma_enable_traffic_jam_assist,
}
for name, setting in settings.items():
  df_fw[f'code_{name}'] = df_fw.apply(get_setting(setting), axis=1)


# Clear settings if platform is Q4
# TODO: simplify
for setting in settings.keys():
  df_fw[f'code_{setting}'] = df_fw.apply(
    lambda row: '' if row['Platform'] == 'Q4' else row[f'code_{setting}'],
    axis=1,
  )

In [ ]:
from panda.python.uds import DATA_IDENTIFIER_TYPE


# Drop columns that we don't care about (not in the properties)
df_fw.drop(
  columns=[col for col in df_nhtsa.columns if col not in properties and col != 'VIN'],
  inplace=True,
)


# Merge 'Series' and 'Trim' properties (they should be mutually exclusive)
if 'Trim' in properties and 'Series' in properties:
  def get_series_or_trim(row):
    series, trim = row['Series'], row['Trim']
    if series and trim:
      raise ValueError(f'{row["VIN"]} Both Series and Trim are set: {series} and {trim}')
    return series or trim


  df_fw['Series'] = df_fw.apply(get_series_or_trim, axis=1)
  df_fw.drop(columns=['Trim'], inplace=True)
  properties.pop('Trim', None)





# Apply filters
constants = {
  # 'ModelYear': '2020',
  # 'DisplacementL': '1.5',
  # 'DriveType': '4x2',
  # 'EngineCylinders': '3',
  # 'Series': 'Titanium',
}
for col, value in constants.items():
  df_fw = df_fw[df_fw[col] == value]
  df_fw.drop(columns=[col], inplace=True)


# Drop columns that are all empty
df_fw = df_fw.loc[:, (df_fw != '').any(axis=0)]


# Drop the VIN
df_fw.drop(columns=['VIN'], inplace=True)


# Sort by columns
extra_columns = list(set(properties.keys()) - {'ModelYear', 'Series'})

df_fw.sort_values(
  by=['ModelYear', 'Series'] + [f'code_{name}' for name in settings.keys()] + extra_columns + [f'{name}_fw' for name in ecus.keys()],
  ascending=False,
  inplace=True,
  ignore_index=True,
)


# Drop columns that are all the same
# df_fw = df_fw.loc[:, df_fw.apply(pd.Series.nunique) != 1]


# Add asterisks to column name where the value is the same for all rows
# df_fw.rename(
#   columns={
#     col: f'*{col}' if len(set(df_fw[col].unique())) == 1 else col
#     for col in df_fw.columns
#   },
#   inplace=True,
# )

# Remove duplicate rows

count = len(df_fw)
df_fw.drop_duplicates(inplace=True)
print(f'Removed {count - len(df_fw)} duplicate rows')


df_fw

In [ ]:
df_fw_analysis = df_fw.copy()

SORTED_TRIM_LEVELS = [
  'Base',
  # Escape, Focus, Fusion
  'S',
  'SE',
  'SEL',
  'Titanium',
  'Active',
  'ST Line',
  'ST Line Select',
  'ST Line Elite',
  'ST Line Premium',
  'PHEV',
  # Bronco Sport
  'BIG BEND',
  'OUTER BANKS',
  'BADLANDS',
  'HERITAGE',
  # Explorer
  'XL',
  'XLT',
  'Limited',
  'ST',
  'Platinum',
  'Timberline',
  'King Ranch',
  # Aviator
  'Standard',
  'Reserve',
  'Livery',
  'Grand Touring',
]

def sort_trim_level(trim):
  if trim in SORTED_TRIM_LEVELS:
    return SORTED_TRIM_LEVELS.index(trim)
  return 1000

for ecu in reversed(ecus.keys()):
  print()
  print(f'# Ecu.{ecu}')

  fw_groups = df_fw_analysis \
    .drop(
      columns=[f'{name}_fw' for name in ecus.keys() if name != ecu] + [f'{name}_part' for name in ecus.keys() if name != ecu],
    ) \
    .rename(
      columns={
        f'{ecu}_fw': 'fw',
        f'{ecu}_part': 'part',
      },
    ) \
    .groupby(
      by=['part', 'fw'],
      dropna=False,
    )
  # print(fw_groups.nunique().to_string())

  if len(fw_groups) > 1:
    for name, group in fw_groups:
      print(name)
      # print(group.groupby(
      #   by=['ModelYear', 'Model', 'Series'] + list(set(group.columns) - {'ModelYear', 'Model', 'Series', 'code_acc', 'code_lca', 'part', 'fw'}) + ['code_acc', 'code_lca'],
      #   dropna=False,
      # ).count().to_string())

      drive_type = group["DriveType"].unique()
      # fuel_type_secondary = group["FuelTypeSecondary"].unique()
      electrification_level = group["ElectrificationLevel"].unique()
      code_acc = group["code_acc"].unique()
      # "Both" if "RadarFusion" in code_acc and ("Off" in code_acc or "" in code_acc) else "Yes" if "RadarFusion" in code_acc else "No"
      code_lca = group["code_lca"].unique()

      # FuelTypePrimary  FuelTypeSecondary  Hybrid
      # Gasoline                            
      # Gasoline         Electric           FHEV
      # Electric         Gasoline           PHEV
      # group["hybrid"] = group["FuelTypeSecondary"].apply(lambda x: "Electric" in x)

      print(f'  Model         : {", ".join(group["Model"].unique())}')
      print(f'  ModelYear     : {", ".join(sorted(group["ModelYear"].astype(str).unique()))}')
      print(f'  Series        : {", ".join(sorted(group["Series"].unique(), key=sort_trim_level))}')
      print(f'  DisplacementL : {", ".join(map(str, sorted(group["DisplacementL"].unique())))}')
      print(f'  4WD?          : {"Both" if "4WD" in drive_type and "RWD" in drive_type else "Yes" if "4WD" in drive_type else "No"}')
      print(f'  Hybrid?       : {", ".join(electrification_level)}')
      print(f'  ACC?          : {", ".join(code_acc)}')
      print(f'  LCA?          : {", ".join(code_lca)}')

      print()

  print()
